<a href="https://colab.research.google.com/github/murillo-borges/ifood-ab-campaign-case/blob/main/Ifood_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparação do ambiente e ETL

### Database Pedidos

In [3]:
# 1 - Instalação e configuração do ambiente
!apt-get install openjdk-11-jdk -y
!pip install pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
print("Ambiente configurado!\n")

# 2 - Iniciando a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("iFood Order Case").getOrCreate()
print("SparkSession criada!\n")

# 3 - Baixando o arquivo order.json.gz
import urllib.request
url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
local_path = "order.json.gz"
urllib.request.urlretrieve(url, local_path)
print("Download concluído!\n")

# 4 - Lendo o arquivo JSON compactado com PySpark
df_order = spark.read.json(local_path, multiLine=False)
print("Dados carregados!\n")

# 5 - Exibindo as primeiras linhas do DataFrame
print("Primeiras linhas do DataFrame:")
df_order.show(5, truncate=False)

# 6 - Exibindo o schema do DataFrame
print("\n Estrutura do DataFrame:")
df_order.printSchema()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre
  x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk
  openjdk-11-jre x11-utils
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 6,920 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-extra all 2.37-2build1 [2,041 kB]
Get:3 http://archive.ubuntu.com/ubuntu jam

### Database Usuários

In [4]:
# 1 - Baixando o arquivo CSV compactado (.gz) da internet
import urllib.request

url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
local_path = "consumer.csv.gz"
urllib.request.urlretrieve(url, local_path)
print("Download concluído!\n")

# 2 - Lendo o arquivo CSV com PySpark
df_consumer = spark.read.option("header", True).option("inferSchema", True).csv(local_path)
print("Arquivo carregado em um DataFrame Spark!\n")

# 3 - Exibindo as primeiras linhas do DataFrame
print("Primeiras linhas do dataset:")
df_consumer.show(5, truncate=False)

# 4 - Exibindo o schema do DataFrame
print("\n Estrutura geral do dataset:")
df_consumer.printSchema()

Download concluído!

Arquivo carregado em um DataFrame Spark!

Primeiras linhas do dataset:
+----------------------------------------------------------------+--------+-----------------------+------+-------------+-------------------+---------------------+
|customer_id                                                     |language|created_at             |active|customer_name|customer_phone_area|customer_phone_number|
+----------------------------------------------------------------+--------+-----------------------+------+-------------+-------------------+---------------------+
|e8cc60860e09c0bb19610b06ced69c973eb83982cfc98e397ce65cba92f70928|pt-br   |2018-04-05 14:49:18.165|true  |NUNO         |46                 |816135924            |
|a2834a38a9876cf74e016524dd2e8c1f010ee12b2b684d58c40ab11eef19b6eb|pt-br   |2018-01-14 21:40:02.141|true  |ADRIELLY     |59                 |231330577            |
|41e1051728eba13341136d67d0757f8d8cc44b2a405b718a8c5efba2c093b2c0|pt-br   |2018-01-07 03:47:1

### Database Merchants - Restaurantes

In [5]:
# 1 - Baixando o arquivo CSV compactado (.gz) da internet
import urllib.request

url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
local_path = "restaurant.csv.gz"
urllib.request.urlretrieve(url, local_path)
print("Download concluído!\n")

# 2 - Lendo o arquivo CSV com PySpark
df_restaurant = spark.read.option("header", True).option("inferSchema", True).csv(local_path)
print("Arquivo carregado em um DataFrame Spark!\n")

# 3 - Exibindo as primeiras linhas do DataFrame
print("Primeiras linhas do dataset:")
df_restaurant.show(5, truncate=False)

# 4 - Exibindo o schema do DataFrame
print("\n Estrutura geral do dataset:")
df_restaurant.printSchema()

Download concluído!

Arquivo carregado em um DataFrame Spark!

Primeiras linhas do dataset:
+----------------------------------------------------------------+-----------------------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+--------------+--------------+----------------+
|id                                                              |created_at             |enabled|price_range|average_ticket|takeout_time|delivery_time|minimum_order_value|merchant_zip_code|merchant_city |merchant_state|merchant_country|
+----------------------------------------------------------------+-----------------------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+--------------+--------------+----------------+
|d19ff6fca6288939bff073ad0a119d25c0365c407e9e5dd999e7a3e53c6d5d76|2017-01-23 12:52:30.91 |false  |3          |60.0          |0           |50           |30.0               |14025            |RIBE

### Database Marcação de usuários que participaram do teste A/B

In [6]:
# 1 - Instalando Java e PySpark no Google Colab
!apt-get install openjdk-11-jdk -y
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# 2 - Iniciando SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("AB Test Case").getOrCreate()

# 3 - Baixando o arquivo .tar.gz
import urllib.request
import tarfile

url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
local_tar_path = "ab_test_ref.tar.gz"
urllib.request.urlretrieve(url, local_tar_path)
print("Download concluído!\n")

# 4 - Extraindo o conteúdo
extract_path = "ab_test_ref_extracted"
os.makedirs(extract_path, exist_ok=True)
with tarfile.open(local_tar_path, "r:gz") as tar:
    tar.extractall(path=extract_path)
print("✅ Extração concluída!\n")

# 5 - Caminho direto para o arquivo extraído
csv_path = os.path.join(extract_path, "ab_test_ref.csv")
print(f"Lendo arquivo: {csv_path}")

# 6 - Definir schema do arquivo
from pyspark.sql.types import StructType, StructField, StringType
schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("is_target", StringType(), True),
])

# 7 - Leitura do CSV com PySpark
df_ab_test = spark.read.option("header", True)\
                       .option("sep", ",")\
                       .schema(schema)\
                       .csv(csv_path)

# 8 - Visualizar os dados
print("Primeiras linhas do DataFrame:")
df_ab_test.show(5, truncate=False)

print("\n Estrutura do DataFrame:")
df_ab_test.printSchema()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.27+6~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Download concluído!

✅ Extração concluída!

Lendo arquivo: ab_test_ref_extracted/ab_test_ref.csv
Primeiras linhas do DataFrame:
+----------------------------------------------------------------+---------+
|customer_id                                                     |is_target|
+----------------------------------------------------------------+---------+
|755e1fa18f25caec5edffb188b13fd844b2af8cf5adedcf77c028f36cb9382ea|target   |
|b821aa8372b8e5b82cdc283742757df8c45eecdd72adf411716e710525d4edf1|control  |
|d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc007f5297541ec40b63b|control  |
|6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c8b535718bbc651ab02|target   |
|dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f5e3763fa657c23a58d|control  |
+-----------------

**Explanation of the fix:**

1.  **`items_schema`**: We define a `StructType` that matches the expected structure of each item within the `items` array. This includes nested `StructType` for fields like `addition`, `discount`, `unitPrice`, `totalValue`, `totalAddition`, and `totalDiscount`, and an `ArrayType` for `garnishItems`. I've used `StringType` for currency values and prices based on the example data in the output of cell `Bh7E-EBFhrMs`, and `LongType` for `sequence`.
2.  **`from_json(col("items"), items_schema)`**: This function parses the JSON string in the `items` column according to the `items_schema` we defined. The result is a new column named `parsed_items` which is of type `ArrayType(StructType(...))`.
3.  **`explode("parsed_items")`**: Now that `parsed_items` is an array type, we can successfully apply the `explode` function to it. This creates a new row for each item in the array.
4.  **`df_exploded.select("order_id", "item.*").show(5, truncate=False)`**: This line selects the original `order_id` and all the fields from the exploded `item` column to show the result of the explosion.

This corrected code will successfully extract the individual items from the `items` column and create a new row for each item, allowing you to analyze the items within each order.

## O Desafio – Análise do teste A/B (ETAPA 1)

### a) Definindo os indicadores relevantes para mensurar o sucesso da campanha e analisar se ela teve impacto significativo dentro do período avaliado:

1. Join dos dados de pedidos com teste A/B e restaurantes


In [8]:
# Base principal de análise
df_ab_orders = df_order.join(df_ab_test, on="customer_id", how="inner")
df_ab_orders = df_ab_orders.join(df_restaurant, df_ab_orders["merchant_id"] == df_restaurant["id"], "inner")

2. KPIs principais por grupo (usuários e pedidos)

In [9]:
from pyspark.sql.functions import col, countDistinct, count, sum, avg

df_metrics = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    count("order_id").alias("qtd_pedidos"),
    sum("order_total_amount").alias("valor_total_pedidos"),
    avg("order_total_amount").alias("ticket_medio")
).withColumn(
    "pedidos_por_usuario", col("qtd_pedidos") / col("qtd_usuarios")
)

3. Retenção de usuários (usuários com >1 pedido)


In [10]:
from pyspark.sql.functions import when, round

df_freq = df_ab_orders.groupBy("is_target", "customer_id").agg(count("order_id").alias("qtd_pedidos"))

df_retencao = df_freq.groupBy("is_target").agg(
    count("*").alias("total_usuarios"),
    count(when(col("qtd_pedidos") > 1, True)).alias("usuarios_retidos")
).withColumn(
    "taxa_retencao", round((col("usuarios_retidos") / col("total_usuarios")) * 100, 2)
).select("is_target", "taxa_retencao")

4. Incremento percentual do ticket médio


In [11]:
from pyspark.sql.functions import first

df_ticket_medio = df_ab_orders.groupBy("is_target").agg(round(avg("order_total_amount"), 2).alias("ticket_medio"))

df_incremento = df_ticket_medio.groupBy().pivot("is_target").agg(first("ticket_medio")).withColumn(
    "incremento_percentual", round(((col("target") - col("control")) / col("control")) * 100, 2)
).select("incremento_percentual")

5. Heavy users (usuários que estão no top 20% de pedidos)

In [12]:
df_user_freq = df_ab_orders.groupBy("customer_id").agg(count("order_id").alias("qtd_pedidos"))
percentil_80 = df_user_freq.approxQuantile("qtd_pedidos", [0.80], 0.01)[0]

df_user_freq = df_user_freq.withColumn(
    "heavy_user", when(col("qtd_pedidos") >= percentil_80, 1).otherwise(0)
)

df_heavy = df_user_freq.join(df_ab_test, on="customer_id", how="inner")

df_heavy_summary = df_heavy.groupBy("is_target").agg(
    count(when(col("heavy_user") == 1, True)).alias("qtd_heavy_users"),
    count("*").alias("total_usuarios")
).withColumn(
    "percentual_heavy_users", round((col("qtd_heavy_users") / col("total_usuarios")) * 100, 2)
).select("is_target", "percentual_heavy_users")

6. Tempo médio de entrega por grupo

In [13]:
df_delivery_time = df_ab_orders.groupBy("is_target").agg(
    round(avg("delivery_time"), 2).alias("tempo_medio_entrega")
)

7. Total de restaurantes por grupo (com base em pedidos)


In [14]:
df_qtd_rest = df_ab_orders.groupBy("is_target").agg(
    countDistinct("merchant_id").alias("qtd_restaurantes")
)

8. Média de pedidos por restaurante

In [15]:
df_avg_pedidos_rest = df_ab_orders.groupBy("is_target", "merchant_id").agg(
    count("order_id").alias("qtd_pedidos_rest")
).groupBy("is_target").agg(
    round(avg("qtd_pedidos_rest"), 2).alias("media_pedidos_por_restaurante")
)

10. Juntando todos os KPIs em uma tabela final

In [28]:
# Juntando os dataframe
df_final = df_metrics \
    .join(df_retencao, on="is_target", how="left") \
    .join(df_heavy_summary, on="is_target", how="left") \
    .join(df_delivery_time, on="is_target", how="left") \
    .join(df_qtd_rest, on="is_target", how="left") \
    .join(df_avg_pedidos_rest, on="is_target", how="left")

#df_final.show(truncate=False)

In [29]:
# Convertendo Spark DataFrame para Pandas DataFrame
df_final_pd = df_final.toPandas()

# Criando novo dataframe
df_final_formatado = pd.DataFrame()

# Aplicando formatação de dados
df_final_formatado["is_target"] = df_final_pd["is_target"]
df_final_formatado["qtd_usuarios"] = df_final_pd["qtd_usuarios"].apply(lambda x: f"{x:,.0f}".replace(",", "."))
df_final_formatado["qtd_pedidos"] = df_final_pd["qtd_pedidos"].apply(lambda x: f"{x:,.0f}".replace(",", "."))
df_final_formatado["valor_total_pedidos"] = df_final_pd["valor_total_pedidos"].apply(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
df_final_formatado["ticket_medio"] = df_final_pd["ticket_medio"].apply(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
df_final_formatado["pedidos_por_usuario"] = df_final_pd["pedidos_por_usuario"].apply(lambda x: f"{x:.1f}".replace(".", ","))
df_final_formatado["taxa_retencao"] = df_final_pd["taxa_retencao"].apply(lambda x: f"{x:.2f}%".replace(".", ","))
df_final_formatado["percentual_heavy_users"] = df_final_pd["percentual_heavy_users"].apply(lambda x: f"{x:.2f}%".replace(".", ","))
df_final_formatado["tempo_medio_entrega"] = df_final_pd["tempo_medio_entrega"].apply(lambda x: f"{x:.2f}".replace(".", ","))
df_final_formatado["qtd_restaurantes"] = df_final_pd["qtd_restaurantes"].apply(lambda x: f"{x:,.0f}".replace(",", "."))
df_final_formatado["media_pedidos_por_restaurante"] = df_final_pd["media_pedidos_por_restaurante"].apply(lambda x: f"{x:.2f}".replace(".", ","))


# Exibindo resultado
print(df_final_formatado.to_string(index=False))

is_target qtd_usuarios qtd_pedidos valor_total_pedidos ticket_medio pedidos_por_usuario taxa_retencao percentual_heavy_users tempo_medio_entrega qtd_restaurantes media_pedidos_por_restaurante
  control      360.542   1.525.576    R$ 73.071.872,88     R$ 47,90                 4,2        74,70%                 21,53%               22,63            7.196                        212,00
   target      445.924   2.136.745   R$ 102.007.569,84     R$ 47,74                 4,8        79,51%                 25,87%               22,66            7.227                        295,66


#### **Análise Comparativa — Grupo Controle vs Grupo Target**

| Indicador                    | Controle  | Target    | Diferença | Interpretação                               |
| ---------------------------- | --------- | --------- | --------- | ------------------------------------------- |
| **Usuários (qtd\_usuarios)** | 360.542   | 445.924   | +85.382   | Target teve uma base maior.                 |
| **Pedidos (qtd\_pedidos)**   | 1.525.576 | 2.136.745 | +611.169  | Target pediu mais.                          |
| **Ticket médio**             | R\$ 47,90 | R\$ 47,74 | -R\$ 0,16 | Leve queda no valor por pedido no target.   |
| **Pedidos por usuário**      | 4,23      | 4,79      | +13,2%    | Mais engajamento no target.                 |
| **Retenção de usuários (%)** | 74,7%     | 79,51%    | +4,8 p.p. | A campanha reteve melhor os usuários.       |
| **Heavy Users (%)**          | 21,53%    | 25,87%    | +4,3 p.p. | Mais usuários muito ativos no target.       |
| **Tempo médio de entrega**   | 22,63 min | 22,66 min | ≈         | Não há impacto logístico relevante.         |
| **Pedidos por restaurante**  | 212,0     | 295,66    | +39,4%    | Restaurantes no grupo target venderam mais. |


**Análise de Engajamento e Produto:**

Observando as métricas de comportamento, a campanha foi um sucesso absoluto.


*   Frequência de Pedidos: Os usuários do grupo target fizeram, em média, 4,79 pedidos, um aumento de 13,2% em relação aos 4,23 do grupo control. Isso mostra que o incentivo funcionou para criar o hábito de pedir mais vezes.

*   Taxa de Retenção: A retenção do grupo target foi de 79,51%, quase 5 pontos percentuais maior que a do grupo control. Este é um ganho massivo e o indicador mais forte de que a campanha gerou lealdade.


*   "Heavy Users": O percentual de usuários de alta frequência subiu de 21,53% para 25,87%. A campanha foi eficaz em converter usuários e engajar a base.


Conclusão de Produto: Sem dúvida, a campanha foi um sucesso em engajar a base de usuários, aumentar a frequência e reter mais clientes.

### b) Análise da Viabilidade Financeira

**Premissas Adotadas:**

*   Custo da Campanha: R$ 10,00 por cada pedido realizado no grupo target.
*   Take Rate (Receita do iFood): 20% sobre o valor total dos pedidos (GMV).


**1 - Custo da Campanha**

In [22]:
from pyspark.sql.functions import col

# --- Premissas ---
CUSTO_CUPOM_POR_PEDIDO = 10.00

# --- Cálculo Direto ---
custo_total_campanha = df_ab_orders.filter(col("is_target") == "target").count() * CUSTO_CUPOM_POR_PEDIDO

# --- Resultado ---
print(f"Custo Total da Campanha: R$ {custo_total_campanha:,.2f}")

Custo Total da Campanha: R$ 21,367,450.00


**2 - Receita Incremental**

In [23]:
from pyspark.sql.functions import col, sum, countDistinct

# --- Premissas ---
# Comissão estimada do Ifood.
TAKE_RATE = 0.20

# --- 1. Agregação ---
# Agrupamos os dados para obter os totais de usuários e GMV para cada grupo.
summary_df = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    sum("order_total_amount").alias("gmv_total")
)

# --- 2. Extração dos Dados Agregados ---
metrics = {row["is_target"]: row for row in summary_df.collect()}

control_metrics = metrics.get("control")
target_metrics = metrics.get("target")

# --- 3. Cálculo Financeiro ---
# Verificamos se ambos os grupos existem antes de calcular
if control_metrics and target_metrics:
    # Comportamento base: Qual foi o GMV por usuário no grupo de controle?
    gmv_por_usuario_control = control_metrics["gmv_total"] / control_metrics["qtd_usuarios"]

    # Projeção: Qual seria o GMV do grupo Target se eles fossem como o Controle?
    gmv_baseline_target = gmv_por_usuario_control * target_metrics["qtd_usuarios"]

    # Receita projetada (Baseline) vs. Receita Real
    receita_baseline_target = gmv_baseline_target * TAKE_RATE
    receita_real_target = target_metrics["gmv_total"] * TAKE_RATE

    # A Receita Incremental é a diferença entre o que aconteceu e o que teria acontecido.
    receita_incremental = receita_real_target - receita_baseline_target
else:
    receita_incremental = 0  # Define como 0 se um dos grupos não existir

# --- 4. Resultado ---
print("CÁLCULO DA RECEITA INCREMENTAL DA CAMPANHA")
print(f"Take Rate (premissa): {TAKE_RATE:.0%}")
print("---------------------------------------------")
print(f"Receita Incremental Gerada: R$ {receita_incremental:,.2f}")

CÁLCULO DA RECEITA INCREMENTAL DA CAMPANHA
Take Rate (premissa): 20%
---------------------------------------------
Receita Incremental Gerada: R$ 2,326,226.29


**3 - Resultado Líquido**

In [24]:
from pyspark.sql.functions import col, sum, count, countDistinct

# --- 1. Premissas da Análise ---
TAKE_RATE = 0.20  # 20% de comissão para o iFood
CUSTO_CUPOM_POR_PEDIDO = 10.00  # R$ 10,00 por pedido no grupo Target

# --- 2. Agregação ---
summary_df = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    count("order_id").alias("qtd_pedidos"),
    sum("order_total_amount").alias("gmv_total")
)

# --- 3. Extração dos Dados para o Modelo Financeiro ---
metrics = {row["is_target"]: row for row in summary_df.collect()}

control_metrics = metrics.get("control")
target_metrics = metrics.get("target")

# --- 4. Cálculo dos Componentes e do Resultado Líquido ---
receita_incremental = 0
custo_total_campanha = 0

if control_metrics and target_metrics:
    # --- Cálculo do Custo da Campanha ---
    custo_total_campanha = target_metrics["qtd_pedidos"] * CUSTO_CUPOM_POR_PEDIDO

    # --- Cálculo da Receita Incremental ---
    gmv_por_usuario_control = control_metrics["gmv_total"] / control_metrics["qtd_usuarios"]
    gmv_baseline_target = gmv_por_usuario_control * target_metrics["qtd_usuarios"]
    receita_baseline_target = gmv_baseline_target * TAKE_RATE
    receita_real_target = target_metrics["gmv_total"] * TAKE_RATE
    receita_incremental = receita_real_target - receita_baseline_target

# --- Cálculo Final ---
resultado_liquido = receita_incremental - custo_total_campanha


# --- 5. Exibição do Resultado ---
print("COMPONENTES DA ANÁLISE:")
print(f"  (+) Receita Incremental Gerada: R$ {receita_incremental:,.2f}")
print(f"  (-) Custo Total da Campanha:    R$ {custo_total_campanha:,.2f}")
print("--------------------------------------------------")
print(f"  📊 Resultado Líquido Final:    R$ {resultado_liquido:,.2f}")

COMPONENTES DA ANÁLISE:
  (+) Receita Incremental Gerada: R$ 2,326,226.29
  (-) Custo Total da Campanha:    R$ 21,367,450.00
--------------------------------------------------
  📊 Resultado Líquido Final:    R$ -19,041,223.71


**4 - ROI: Retorno sobre Investimento**

In [25]:
from pyspark.sql.functions import col, sum, count, countDistinct

# --- 1. Premissas da Análise ---
TAKE_RATE = 0.20  # 20% de comissão para o iFood
CUSTO_CUPOM_POR_PEDIDO = 10.00  # R$ 10,00 por pedido no grupo Target

# --- 2. Agregação ---
summary_df = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    count("order_id").alias("qtd_pedidos"),
    sum("order_total_amount").alias("gmv_total")
)

# --- 3. Extração dos Dados para o Modelo Financeiro ---
metrics = {row["is_target"]: row for row in summary_df.collect()}

control_metrics = metrics.get("control")
target_metrics = metrics.get("target")

# --- 4. Cálculo dos Componentes Financeiros ---

receita_incremental = 0
custo_total_campanha = 0
resultado_liquido = 0
roi = 0

if control_metrics and target_metrics:
    # --- Custo da Campanha ---
    custo_total_campanha = target_metrics["qtd_pedidos"] * CUSTO_CUPOM_POR_PEDIDO

    # --- Receita Incremental ---
    gmv_por_usuario_control = control_metrics["gmv_total"] / control_metrics["qtd_usuarios"]
    gmv_baseline_target = gmv_por_usuario_control * target_metrics["qtd_usuarios"]
    receita_baseline_target = gmv_baseline_target * TAKE_RATE
    receita_real_target = target_metrics["gmv_total"] * TAKE_RATE
    receita_incremental = receita_real_target - receita_baseline_target

    # --- Resultado Líquido ---
    resultado_liquido = receita_incremental - custo_total_campanha

    # --- Cálculo Final do ROI ---
    # Verificamos se o custo é maior que zero para evitar divisão por zero
    if custo_total_campanha > 0:
        roi = resultado_liquido / custo_total_campanha

# --- 5. Exibição do Relatório Financeiro Completo ---
print(f"  📈 ROI (Retorno sobre Investimento): {roi:.2%}")

  📈 ROI (Retorno sobre Investimento): -89.11%


**Conclusão: A Campanha Teve Retorno?**


Não. A campanha **não teve retorno financeiro positivo.**

**Apesar de gerar:**

*   Maior engajamento dos usuários (mais pedidos por usuário)
*   Maior taxa de retenção
*   Mais heavy users
*   Aumento nas vendas dos restaurantes




O custo da campanha (desconto) foi quase 10x maior que a receita adicional gerada, gerando **prejuízo** de mais de R$ 19 milhões e um ROI negativo de -89,11%.


💡 Insight:

*   A estratégia melhorou comportamento e engajamento, mas não foi eficiente financeiramente.
*   Se o cupom fosse menor (ex: R$5) ou vinculado a pedidos acima de um valor mínimo (ex: R\$ 60.00), poderia manter os efeitos positivos com menor custo.





### c) Recomendações de oportunidades de melhoria nessa ação e nova proposta de teste A/B para validar essas hipóte-ses.

### **Resumo da campanha atual**

A campanha com cupom aumentou o engajamento:

* Retenção subiu para **79,5%** (vs 74,7%)
* Mais **pedidos por usuário** (4,79 vs 4,23)
* Mais **heavy users** (25,8% vs 21,5%)

❌ **Problema**: o custo foi alto demais
**Resultado líquido**: –R\$ 19 milhões
📉 **ROI**: –89.11%

---

##  **Oportunidades de melhoria**

1. **Focar em usuários inativos ou novos**, não em todos.
2. **Reduzir o valor do cupom** ou exigir **valor mínimo de compra**.
3. **Estimular o aumento do ticket médio** com frete grátis.

---

##  **Nova proposta de teste A/B**

Analisei 3 variáveis que poderiam ser melhoradas, e , por esse motivo decidi inserir 3 propostas de teste A/B com essas 3 variáveis diferentes:


| Grupo    | Estratégia                          |
| -------- | ----------------------------------- |
| Controle | Sem cupom                           |
| Teste A  | R\$10 só para usuários inativos  |

---

| Grupo    | Estratégia                          |
| -------- | ----------------------------------- |
| Controle | Sem cupom                           |
| Teste B  | R\$5 de desconto em pedidos > R\$40     |


---


| Grupo    | Estratégia                          |
| -------- | ----------------------------------- |
| Controle | Sem cupom                           |
| Teste C  | Frete Grátis para pedidos acima de R$ 65,00         |

---

**Métricas para analisar após teste**: Receita incremental, Resultado Líquido, ROI, Retenção, Reativação, Ticket médio.


## O Desafio – Análise por Segmentação (ETAPA 2)

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, avg, when, percent_rank, round as spark_round
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("SegmentABTest_NoDate").getOrCreate()

# 2. CRIAÇÃO DA BASE DE ANÁLISE POR USUÁRIO

# Agregamos os dados para ter uma visão única por cliente.
print("--- Passo 1: Criando a base de análise por usuário... ---")
df_user_features = df_ab_orders.groupBy("customer_id", "is_target").agg(
    count("order_id").alias("total_pedidos"),
    avg("order_total_amount").alias("ticket_medio_usuario"),
    sum("order_total_amount").alias("gasto_total_usuario")
)

# 3. APLICAÇÃO DOS CRITÉRIOS DE SEGMENTAÇÃO
# Adicionamos as "etiquetas" de segmento a cada usuário.
print("--- Passo 2: Aplicando os critérios de segmentação... ---")

# --- Segmentação por Frequência ---
window_freq = Window.partitionBy("is_target").orderBy("total_pedidos")
df_user_features = df_user_features.withColumn("rank_freq", percent_rank().over(window_freq))
df_user_features = df_user_features.withColumn("segmento_frequencia",
    when(col("rank_freq") >= 0.8, "1. Heavy User (Top 20%)")
    .when((col("rank_freq") >= 0.3) & (col("rank_freq") < 0.8), "2. Casual (30-80%)")
    .otherwise("3. Leve (Bottom 30%)")
)

# --- Segmentação por Valor (Ticket Médio) ---
window_valor = Window.partitionBy("is_target").orderBy("ticket_medio_usuario")
df_user_features = df_user_features.withColumn("rank_valor", percent_rank().over(window_valor))
df_user_features = df_user_features.withColumn("segmento_valor",
    when(col("rank_valor") >= 0.5, "1. Alto Valor")
    .otherwise("2. Baixo Valor")
)


# 4. ANÁLISE CRUZADA FINAL: KPIs POR SEGMENTO
# Agrupamos por grupo de teste E por segmento para gerar os relatórios.
print("--- Passo 3: Gerando as análises cruzadas por segmento... ---")


print("\n\n" + "="*70)
print("          (c) ANÁLISE DE RESULTADOS POR SEGMENTO DE FREQUÊNCIA")
print("="*70)
df_analise_frequencia = df_user_features.groupBy("is_target", "segmento_frequencia").agg(
    count("customer_id").alias("qtd_usuarios"),
    spark_round(avg("total_pedidos"), 2).alias("pedidos_por_usuario"),
    spark_round(avg("ticket_medio_usuario"), 2).alias("ticket_medio_segmento")
).orderBy("segmento_frequencia", "is_target")
df_analise_frequencia.show(truncate=False)


print("\n" + "="*70)
print("          (c) ANÁLISE DE RESULTADOS POR SEGMENTO DE VALOR (TICKET MÉDIO)")
print("="*70)
df_analise_valor = df_user_features.groupBy("is_target", "segmento_valor").agg(
    count("customer_id").alias("qtd_usuarios"),
    spark_round(avg("total_pedidos"), 2).alias("pedidos_por_usuario"),
    spark_round(avg("ticket_medio_usuario"), 2).alias("ticket_medio_segmento")
).orderBy("segmento_valor", "is_target")
df_analise_valor.show(truncate=False)


--- Passo 1: Criando a base de análise por usuário... ---
--- Passo 2: Aplicando os critérios de segmentação... ---
--- Passo 3: Gerando as análises cruzadas por segmento... ---


          (c) ANÁLISE DE RESULTADOS POR SEGMENTO DE FREQUÊNCIA
+---------+-----------------------+------------+-------------------+---------------------+
|is_target|segmento_frequencia    |qtd_usuarios|pedidos_por_usuario|ticket_medio_segmento|
+---------+-----------------------+------------+-------------------+---------------------+
|control  |1. Heavy User (Top 20%)|61450       |13.25              |47.92                |
|target   |1. Heavy User (Top 20%)|77251       |14.41              |47.82                |
|control  |2. Casual (30-80%)     |92523       |4.21               |48.59                |
|target   |2. Casual (30-80%)     |153193      |4.46               |47.96                |
|control  |3. Leve (Bottom 30%)   |206569      |1.56               |47.78                |
|target   |3. Leve (Bottom 30

### **Análise Estratégica do Teste A/B por Segmentos de Usuário**

A análise inicial do Teste A/B nos mostrou que apesar do resultado financeiro da campanha ser péssimo, foi um sucesso em aumentar a frequência de pedidos e a retenção de clientes. No entanto, esse resultado é uma média geral. A segmentação nos permite aprofundar a análise para entender *quais perfis de clientes* foram mais impactados e *por que*, transformando dados em inteligência para ações futuras mais eficazes e personalizadas.

---

### **a) Definição das Segmentações Propostas**

Para analisar o impacto da campanha de cupons, que é um incentivo de frequência, definimos duas segmentações principais baseadas no comportamento de compra do usuário:

1.  **Segmentação por Frequência de Pedidos:** Agrupa os usuários com base no seu nível de engajamento (quantas vezes eles pedem).
2.  **Segmentação por Perfil de Gasto (Ticket Médio):** Agrupa os usuários com base no valor médio que costumam gastar em cada pedido.

---

### **b) Critérios de Criação e Racional de Cada Segmento**

Aqui detalhamos como cada segmento é construído e qual pergunta de negócio ele visa responder.

#### **1. Segmentação por Frequência de Pedidos**

* **Critérios de Criação:**
    * Calculamos o número total de pedidos de cada usuário.
    * Utilizamos **percentis** para classificar a base de clientes em três grupos distintos:
        * **Heavy Users (Top 20%):** Seus clientes mais leais e frequentes.
        * **Usuários Casuais (Intermediários 50%):** Clientes que utilizam o serviço com regularidade.
        * **Usuários Leves (Bottom 30%):** Clientes novos ou que pedem muito esporadicamente.
* **Racional Estratégico:**
    * A campanha claramente aumentou a frequência geral. O racional desta segmentação é descobrir a **origem desse crescimento**. Queremos responder: **"A campanha foi mais eficaz para criar novos hábitos em clientes pouco engajados ou para acelerar ainda mais os clientes que já eram leais?"**. A resposta nos diz se a campanha é uma ferramenta de **ativação** ou de **fidelização acelerada**, o que impacta diretamente o ROI de cada grupo.

#### **2. Segmentação por Perfil de Gasto (Ticket Médio)**

* **Critérios de Criação:**
    * Calculamos o valor do ticket médio de cada usuário.
    * Utilizamos a **mediana** como ponto de corte para criar dois grupos:
        * **Clientes de Alto Valor:** Usuários que gastam, em média, mais do que a metade da base de clientes.
        * **Clientes de Baixo Valor:** Usuários que gastam, em média, menos do que a metade da base.
* **Racional Estratégico:**
    * Vimos que o ticket médio geral não mudou. O racional aqui é investigar se a campanha teve um apelo diferente para perfis de gasto distintos. A pergunta a ser respondida é: **"O cupom de frequência atraiu mais os 'caçadores de promoção' (focados em economia) ou também engajou clientes que já possuem um ticket médio mais alto?"**. Isso nos ajuda a entender a sensibilidade de cada perfil ao incentivo e o impacto na rentabilidade.

---

### **c) Análise (Hipotética) dos Resultados do Teste por Segmento**

Após rodar o código que está logo abaixo, obtemos os dados que estão nas tabelas para cada segmento. Abaixo, apresento a **análise dos insights**.

#### **Análise por Segmento de FREQUÊNCIA**

| Segmento | Grupo | Pedidos por Usuário (Lift da Campanha) |
| :--- | :--- | :--- |
| **Heavy User** | Controle | 13.25 |
| | Target | 14.41 **(+8.7%)** |
| **Usuário Casual** | Controle | 4.21 |
| | Target | 4.46 **(+6%)** |
| **Usuário Leve** | Controle | 1.56 |
| | Target | 1.58 **(+1.3%)** |

* **Insight:** O maior efeito (+8.7%) foi sobre os **Usuários Heavy User**, seguido pelos Casuais. Nos usuário leves, o efeito foi marginal.
* **Recomendação Estratégica:** A campanha é uma **ferramenta espetacular de ativação e criação de hábito**. O investimento em cupons se justifica muito mais em clientes de baixa frequência, onde o potencial de mudança de comportamento é maior. Para usuário leves, o custo pode não valer a pena, e outras estratégias de reconhecimento (benefícios não-financeiros, exclusividade) podem ser mais eficazes.

#### **Análise por Segmento de GASTO (Ticket Médio)**

| Segmento | Grupo | Pedidos por Usuário |  |
| :--- | :--- | :--- | :--- |
| **Alto Valor** | Controle | 4.4 |
| | Target | 4.9 **(+11%)** |
| **Baixo Valor** | Controle | 4.0
| | Target | 4.6 **(+15%)** |

* **Insight:** O incentivo foi mais atraente para o segmento de **Baixo Valor**, que mostrou um lift maior na frequência de pedidos. No entanto, é exatamente nesse grupo que a rentabilidade é pior, pois o valor fixo do cupom representa uma porcentagem muito maior do valor do pedido, tornando o prejuízo por pedido maior.
* **Recomendação Estratégica:** O cupom de valor fixo não é eficaz para engajar o público de "Baixo Valor" de forma rentável. Para este segmento, seria mais inteligente testar outros tipos de incentivos, como **"Frete Grátis acima de R\$X"** ou **"15% de desconto em pedidos acima de R\$Y"**, que incentivam o aumento do ticket médio e se alinham melhor ao seu perfil de gasto.

---

### **Conclusão Final:**

Os resultados mostram que tratar todos os clientes do mesmo jeito não funciona. Ao analisar por perfil, vimos que faz muito mais sentido trocar a campanha única por várias ações personalizadas. Isso ajuda a investir melhor e a crescer de forma mais inteligente e com mais lucro.

## O Desafio – Próximos Passos Ifood (ETAPA 3)


---

### **3. Próximos Passos e Recomendações Estratégicas**


#### **Resumo Executivo | Data Analysis - iFood**

A Nossa análise aprofundada da campanha de cupons revelou uma dualidade crítica: enquanto a iniciativa foi um sucesso em engajar e reter nossa base de clientes já ativa, ela se mostrou financeiramente insustentável.

O problema foi usar a mesma estratégia para todo mundo, sem considerar as diferenças entre os públicos. Por isso, sugerimos mudar essa abordagem: em vez de uma solução única, usar estratégias diferentes para cada segmento. As recomendações abaixo visam otimizar o investimento, focar em ativação real e transformar a alavanca de frequência em um motor de crescimento lucrativo, com um impacto financeiro positivo estimado em milhões de reais por ciclo de campanha.


---

#### **Diagnóstico Estratégico (O que Aprendemos)**

1. A campanha funcionou como um acelerador para quem já era fiel, não como uma forma de ativar novos usuários. O incentivo de frequência deu resultado entre quem já tinha o hábito de pedir (os clientes mais ativos e os casuais). Mas para os usuários mais inativos, quase não fez diferença — o comportamento deles praticamente não mudou.
2. Usar o mesmo incentivo para todo mundo foi ineficiente e caro. Como a campanha tratou todos os clientes do mesmo jeito, ela acabou gastando muito com quem não respondeu ao cupom, tornando o ROI geral da campanha massivamente negativo (-89% com a premissa de R$10 por cupom).
3. Cada tipo de cliente precisa de um tipo de incentivo diferente. O cupom teve um efeito parecido em quem gasta pouco e em quem gasta muito, mas o lucro que cada um gera é bem diferente. Isso mostra que não adianta usar só um tipo de cupom — precisamos de incentivos mais bem pensados, de acordo com o perfil de cada cliente.

---

### **Plano de Ação: Próximos Passos Recomendados**

Propomos uma abordagem em três frentes, agindo de forma imediata para otimizar custos e de forma estratégica para construir um crescimento sustentável.

#### **Ação 1: Otimização Imediata - Focar o Investimento Onde Funciona**

* Ação: Pausar imediatamente a oferta do cupom de frequência para o segmento de "Usuários Leves".
* Racional: Nossa análise mostrou que os pedidos aumentaram só 1,3% entre esses clientes. Com um resultado tão pequeno, continuar usando essa campanha para esse público acaba sendo um mau negócio.
* **Previsão de Impacto:**
    * Financeiro (Economia Direta): Com base nos dados do teste, o segmento de "Usuários Leves" no grupo target continha ~215 mil usuários que fizeram ~340 mil pedidos. Com um custo de R$ 10 por cupom, essa ação representa uma **economia imediata de R$ 3,4 milhões de reais** por ciclo de campanha, eliminando um investimento com
retorno praticamente nulo.

#### **Ação 2: Nova Estratégia de Ativação (Para "Usuários Leves")**

* Ação: Desenhar e testar uma nova campanha focada exclusivamente na ativação deste segmento, já que o incentivo de frequência não funcionou.

* **Proposta de Novo Teste A/B:**
    * o	Hipótese: "Para usuários com poucos pedidos, um incentivo de alto valor na próxima compra é mais eficaz para quebrar a barreira inicial do que um pequeno incentivo de frequência."
    * **Grupos:**
        * **Grupo A (Controle):** Usuários Leves sem nenhuma ação.
        * **Grupo B (Variante):** Oferecer **"Frete Grátis nos próximos 2 pedidos"**.
        * **Grupo C (Variante):** Oferecer "10% de desconto no próximo pedido acima de R$ 50".
    * **Métrica de Sucesso:** Taxa de conversão para o segundo e terceiro pedido.
* **Previsão de Impacto:**
    * **Estratégico:** Aumentar a taxa de ativação de "Usuários Leves" para "Usuários Casuais" em **15%**. Cada usuário ativado representa um aumento significativo no LTV (Valor do Ciclo de Vida do Cliente), transformando um segmento de baixo valor em uma base de clientes recorrentes.

#### **Ação 3: Estratégia de Rentabilização (Para "Heavy Users" e "Casuais")**

* **Ação:** Otimizar a campanha para os segmentos que responderam bem, com o objetivo de **torná-la lucrativa**.
* **Proposta de Novo Teste A/B:**
    * **Hipótese:** "É possível manter um lift de frequência positivo nestes segmentos com um custo de incentivo menor, alcançando um ROI positivo."
    * **Grupos:**
        * **Grupo A (Controle):** Sem cupom.
        * **Grupo B (Variante):** Cupom de **R$ 4,00** para o próximo pedido.
        * **Grupo C (Variante):** Sistema de gamificação com acúmulo de pontos que resultem em um benefício equivalente a ~R$ 3,00 por pedido.
    * **Métrica de Sucesso:** **Margem de Contribuição por Usuário** e **ROI da Campanha**.
* **Previsão de Impacto:**
    * **Financeiro:** Transformar uma iniciativa com ROI de -89% em uma com **ROI positivo (meta: +20%)**. Isso não só manteria o engajamento desses clientes valiosos, mas transformaria a campanha de um centro de custo em um **motor de crescimento lucrativo**.

---

### **Melhorias no Processo de Testes A/B**

Para garantir que não repitamos os mesmos erros, proponho duas melhorias em nosso processo de experimentação:

1.  **Segmentação como Padrão:** Toda análise de teste A/B deve, por padrão, incluir uma análise segmentada pós-teste para entender os efeitos em diferentes perfis de usuário.
2.  **Métricas Financeiras como Critério:** Todo teste deve ter, além de uma métrica de produto primária (ex: frequência), uma **métrica de "guarda" financeira** (ex: Margem de Contribuição ou ROI). O sucesso de um teste só será declarado se a métrica de produto subir **sem prejudicar** a métrica financeira.

Ao adotar estes próximos passos, o iFood evoluirá de uma estratégia de crescimento baseada em volume para um modelo sofisticado, segmentado e focado em **crescimento rentável e sustentável**.